# chatGPT

today I refactored my code using chatGPT.
here is my original code and its result.

In [ ]:
import re
from google.cloud import storage


class ValidateInput:

    def __init__(self, action, url_from, url_to, project_id=None):
        self.action = action
        self.url_from = url_from
        self.url_to = url_to
        self.sc = storage.Client() if project_id is None else storage.Client(project=project_id)

    def get_bucket_address(self, url):
        return url if url.startswith('gs://') else "gs://" + url

    def get_bucket_name(self, url):
        url = self.get_bucket_address(url)
        bucket_name = re.search(r'gs://([A-Za-z_0-9.-]+)/', url)
        return bucket_name.group(1)

    def get_prefix(self, url):
        url = self.get_bucket_address(url)
        bucket_name = self.get_bucket_name(url)
        prefix = re.search(fr'gs://{bucket_name}/([A-Za-z_0-9.-/]+)', url)
        return prefix.group(1)

    def get_bucket_list(self):
        storage_client = self.sc
        bucket_list = storage_client.list_buckets()
        list_all = []
        for bucket in bucket_list:
            bucket_name = bucket.name  # str(bucket).split(": ")[1].replace(">",'') #my.park
            list_all.append(bucket_name)

        storage_client.close()
        return list_all

    def validate_bucket_name(self):
        bucket_list = self.get_bucket_list()
        if self.get_bucket_name(self.url_from) in bucket_list:
            if self.get_bucket_name(self.url_to) in bucket_list:
                print("valid bucket name for both urls")
                return True
            else:
                print(f"incorrect bucket name : {self.url_to}")
                return False
        else:
            print(f"incorrect bucket name : {self.url_from}")
            return False

    def check_prefix_list(self, url):
        storage_client = self.sc
        prefix_search_result = []

        for blob in storage_client.list_blobs(self.get_bucket_name(url), prefix=self.get_prefix(url)):
            prefix_search_result.append(blob.name)
            storage_client.close()  # is this right?
            if prefix_search_result != []:
                return True
            else:
                return False

    def validate_prefix(self):
        if self.validate_bucket_name():
            storage_client = self.sc

            if self.check_prefix_list(self.url_from) and self.check_prefix_list(self.url_to):
                storage_client.close()
                print("nice urls")
                return True
            else:
                if self.check_prefix_list(self.url_from):
                    print(f"your prefix {self.url_to}  is wrong")
                    return False
                else:
                    print(f"your prefix {self.url_from} is wrong")
                    return False

    def validate_input(self):
        if self.validate_prefix():
            print('you may use this info')
            return True
        else:
            print('loser')
            return False

    def __str__(self):
        # return f"check your bucket address \n adderss from : {self.get_bucket_address(self.url_from)} \n address to : {self.get_bucket_address(self.url_to)}"
        return f"check your bucket name \n bucket from : {self.get_bucket_name(self.url_from)} \n bucket to : {self.get_bucket_name(self.url_to)}"

In [ ]:
import re
from google.cloud import storage


class ValidateInput:

    def __init__(self, action, url_from, url_to, project_id=None):
        self.action = action
        self.url_from = url_from
        self.url_to = url_to
        self.sc = storage.Client() if project_id is None else storage.Client(project=project_id)

    def get_bucket_address(self, url):
        return url if url.startswith('gs://') else "gs://" + url

    def get_bucket_name(self, url):
        url = self.get_bucket_address(url)
        bucket_name = re.search(r'gs://([A-Za-z_0-9.-]+)/', url)
        return bucket_name.group(1)

    def get_prefix(self, url):
        url = self.get_bucket_address(url)
        bucket_name = self.get_bucket_name(url)
        prefix = re.search(fr'gs://{bucket_name}/([A-Za-z_0-9.-/]+)', url)
        return prefix.group(1)

    def get_bucket_list(self):
        storage_client = self.sc
        bucket_list = storage_client.list_buckets()

        for bucket in bucket_list:
            bucket_name = bucket.name
            yield bucket_name

        storage_client.close()

    def validate_bucket_name(self):
        bucket_list = self.get_bucket_list()
        if self.get_bucket_name(self.url_from) in bucket_list:
            if self.get_bucket_name(self.url_to) in bucket_list:
                return True
            else:
                return False
        else:
            return False

    def check_prefix_list(self, url):
        storage_client = self.sc
        prefix_search_result = []

        for blob in storage_client.list_blobs(self.get_bucket_name(url), prefix=self.get_prefix(url)):
            prefix_search_result.append(blob.name)
            storage_client.close()
            if prefix_search_result != []:
                return True
            else:
                return False

    def validate_prefix(self):
        if self.validate_bucket_name():
            storage_client = self.sc

            if self.check_prefix_list(self.url_from) and self.check_prefix_list(self.url_to):
                storage_client.close()
                return True
            else:
                return False
        else:
            return False

    def validate_input(self):
        if self.validate_prefix():
            return True
        else:
            return False

    def __str__(self):
        return f"check your bucket name \n bucket from : {self.get_bucket_name(self.url_from)} \n bucket to : {self.get_bucket_name(self.url_to)


# dataclasses

In [ ]:
user = {
    'name':'Steve',
    'last_name':'Jobs'
}
print(user['name'])

class Person:
    def __init__(self, name, last_name) -> None:
        self.name =name
        self.last_name = last_name

user = Person(name='Steve', last_name='Jobs')
print(user)

In [ ]:
from dataclasses import dataclass

@dataclass
class Person:
    name:str
    last_name:str

user = Person(name='Steve', last_name='Jobs')
print(user)



## positional argument vs keyword argument

In [ ]:
Person('Steve','Jobs') #positional : easy to forget order
Person(name='Steve',last_name='Jobs') #keyword


In [ ]:
# if you want to create class using only keword arguments
from dataclasses import dataclass
#kw_only -> 3.10 이상
@dataclass(kw_only=True)
class Person:
    name:str
    last_name:str

Person(name='Steve', last_name='Jobs') #this will work
# Person('Steve','Jobs') # error

In [ ]:
# immutable
from dataclasses import dataclass
#kw_only -> 3.10 이상
@dataclass(kw_only=True, frozen=True) 
class Person:
    name:str
    last_name:str

Person(name='Steve', last_name='Jobs') #this will work
user.last_name = 'wozniak' #error

In [ ]:
from dataclasses import dataclass
#kw_only -> 3.10 이상
@dataclass(frozen=True, kw_only=True)
class Person:
    name:str
    last_name:str
    rich:bool = True #고급 기본값 지정

user = Person(name='Steve', last_name='Jobs') 
print(user)

In [ ]:
from dataclasses import dataclass, field
from datetime import datetime

def get_time() -> str:
    return f"{datetime.now()}"


@dataclass(frozen=True, kw_only=True)
class Person:
    name:str
    last_name:str
    rich:bool = True #고급 기본값 지정
    created_at:str = field(default_factory=get_time)

user = Person(name='Steve', last_name='Jobs') 
print(user)

In [ ]:
#field 사용하여 목록 속성을 기본값 지정
from dataclasses import dataclass, field
from datetime import datetime

def get_time() -> str:
    return f"{datetime.now()}"

@dataclass(frozen=True, kw_only=True)
class Person:
    name:str
    last_name:str
    rich:bool = True #고급 기본값 지정
    created_at:str = field(default_factory=get_time, repr=False) #repr 숨기기
    companies:list[str] = field(default_factory=list)

user = Person(name='Steve', last_name='Jobs') 
print(user)

In [ ]:
#post__init__
#field 사용하여 목록 속성을 기본값 지정
from dataclasses import dataclass, field


@dataclass(kw_only=True)
class Person:
    first_name:str
    last_name:str
    full_name:str = field(init=False)

    def __post_init__(self):
        self.full_name = f"{self.first_name} {self.last_name}"

user = Person(first_name='Steve', last_name='Jobs') 
print(user.full_name)